<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training_vgg16_bn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil

In [40]:
%%time

!git clone -l -s git://github.com/kavyajeetbora/EAST.git cloned-repo
%cd cloned-repo
!ls

Cloning into 'cloned-repo'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 139 (delta 29), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (139/139), 6.06 MiB | 13.55 MiB/s, done.
Resolving deltas: 100% (74/74), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned-repo
dataset.py		    EAST_resnet.py		  model.py
detect.py		    EAST_training.ipynb		  model_resnet.py
dsiplay_ground_truth.ipynb  EAST_training_vgg16_bn.ipynb  train.py
EAST_detection.ipynb	    loss.py
CPU times: user 80.7 ms, sys: 411 ms, total: 492 ms
Wall time: 6.02 s


In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from dataset import *

In [43]:
# AI4Bharat dataset
image_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Image/23.zip (Unzipped Files)'
label_address = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotation/23_labels.zip (Unzipped Files)'

print('Total image files',len(os.listdir(image_address)))
print('Total annotation files',len(os.listdir(label_address)))

training_dataset = custom_dataset(image_address, label_address)
print(len(training_dataset))

Total image files 4669
Total annotation files 4669
4669


## Dataset Initial Check

In [44]:
img = Image.open(image_address+'/0.jpg')

with open(label_address+'/0.txt', 'r') as f:
			lines = f.readlines()
vertices, labels = extract_vertices(lines)

img, vertices = adjust_height(img, vertices) 
img, vertices = rotate_img(img, vertices)
img, vertices = crop_img(img, vertices, labels, 512)

for vertice in vertices:
  print(vertice)
  print('-'*20)
  
print(labels)

[  9.54030341 237.58617344  50.39888396 234.1837632   52.63949558
 261.09063332  11.78091503 264.49304355]
--------------------
[ 12.36181433 271.46889877  66.17555457 266.98767554  67.75228126
 285.9221397   13.93854103 290.40336293]
--------------------
[164.0799742  370.21908848 185.92110498 367.39685419 191.89751601
 415.06531873 169.05983448 417.97053863]
--------------------
[  4.22199042 294.22287132 126.79773207 284.01564062 131.776869
 343.80868533   9.20112735 354.01591603]
--------------------
[1 1 1 1]


In [45]:
score_map, geo_map, ignored_map = get_score_geo(img, vertices, labels, 0.25, 512)

torch.sum(score_map)

tensor(248.)

## Loss for one batch

In [46]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=8)

imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))

print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())

print(torch.sum(gt_score))

torch.Size([8, 3, 512, 512]) torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128]) torch.Size([8, 1, 128, 128])
tensor(2296.)
CPU times: user 4.42 s, sys: 4.85 s, total: 9.27 s
Wall time: 9.03 s


In [47]:
%%time

from model import EAST
model = EAST()
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

torch.Size([8, 1, 128, 128]) torch.Size([8, 5, 128, 128])
CPU times: user 26.8 s, sys: 1.35 s, total: 28.1 s
Wall time: 15.4 s


In [48]:
from loss import Loss

loss_fn = Loss()

loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.96363068, angle loss is 0.79327667, iou loss is 3.97609377


tensor(12.8725, grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
model = EAST()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
pretrained_path = '/content/drive/My Drive/Colab Notebooks/padh.ai.notebooks/15. Object Detection/EAST scene text detector/model_epoch_295.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter//2, gamma=0.1)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))
      
      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))
    return loss_curve

In [0]:
%%time
loss_curve = train(model, image_address, label_address, batch_size=12, lr=1e-4, epoch_iter=30, interval = 5)

4669


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


classify loss is 0.44646382, angle loss is 0.11542867, iou loss is 0.87991768
Epoch is [1/30], mini-batch is [1/389], time consumption is 2.49065447, batch_loss is 2.48066831
classify loss is 0.29561764, angle loss is 0.00705511, iou loss is 0.48035952
Epoch is [1/30], mini-batch is [2/389], time consumption is 0.99447250, batch_loss is 0.84652823
classify loss is 0.33798367, angle loss is 0.00408785, iou loss is 0.53491074
Epoch is [1/30], mini-batch is [3/389], time consumption is 1.01365590, batch_loss is 0.91377294
classify loss is 0.37024719, angle loss is 0.01903320, iou loss is 0.60252839
Epoch is [1/30], mini-batch is [4/389], time consumption is 1.00876021, batch_loss is 1.16310763
classify loss is 0.26522923, angle loss is 0.00605635, iou loss is 0.48139161
Epoch is [1/30], mini-batch is [5/389], time consumption is 1.05793619, batch_loss is 0.80718440
classify loss is 0.29451060, angle loss is 0.01277997, iou loss is 0.56283855
Epoch is [1/30], mini-batch is [6/389], time co

In [0]:
plt.plot(loss_curve)

[Adam and AdamW algorithms](https://www.fast.ai/2018/07/02/adam-weight-decay/)